In [1]:
import numpy as np
from scipy import stats
from scipy.spatial.distance import pdist, squareform
import pandas as pd
from sklearn.datasets import load_wine
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
import seaborn as sns
from IPython.display import display
pd.set_option('max_rows', 5)
pd.set_option('max_columns', 9)
%matplotlib inline

## k近傍法 (k-nearest neighbors)
---
特徴空間内で距離的に近い教師データに基づいて分類するモデル。

In [2]:
iris = sns.load_dataset('iris')[['sepal_length', 'sepal_width', 'species']]
iris['species'] = pd.Categorical(iris['species'])
print('iris')
display(iris)

iris


,sepal_length,sepal_width,species
0,5.1,3.5,setosa
1,4.9,3.0,setosa
...,...,...,...
148,6.2,3.4,virginica
149,5.9,3.0,virginica


In [3]:
from helpers.knn import algorithm
algorithm.show(iris.iloc[:, :2].values, iris['species'].cat.codes)

interactive(children=(IntSlider(value=3, continuous_update=False, description='k', max=10, min=1), Output()), …

## 仕組み
---
- 分類したいデータから特徴空間内での距離の近いk個の教師データの多数決で分類を決定
- 訓練データからパラメーターを学習せず、訓練データを暗記する怠惰学習(lazy learner)の代表例
- 学習にかかるコストはないが、訓練データが増えると記憶容量・計算量が問題になる
- 特徴が増えるに従って、次元の呪い (計算量が爆発的に増えたり、精度が極端に悪化すること) に陥りやすい

###### 練習問題

`wine`データセットのサンプル間距離を算出 (`scipy.spatial.distance.pdist`と`scipy.spatial.distance.squareform`を使用) し、それぞれのサンプルから最も近い 3 サンプルの`target`ラベルに基づいて、予測クラスを作成する。

In [4]:
loader = load_wine()
wine = pd.DataFrame(np.column_stack([loader.data, loader.target]), columns=loader.feature_names + ['target'])
print('wine')
display(wine)

wine


,alcohol,malic_acid,ash,alcalinity_of_ash,...,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,...,1.04,3.92,1065.0,0.0
1,13.20,1.78,2.14,11.2,...,1.05,3.40,1050.0,0.0
...,...,...,...,...,...,...,...,...,...
176,13.17,2.59,2.37,20.0,...,0.60,1.62,840.0,2.0
177,14.13,4.10,2.74,24.5,...,0.61,1.60,560.0,2.0


In [5]:
pdist??

In [6]:
squareform??

In [7]:
dist_matrix = squareform(pdist(wine.iloc[:, :-1].values))
nearests3 = pd.DataFrame(np.argsort(dist_matrix, axis=1)[:, :3])
pred = stats.mode(nearests3.applymap(lambda i: wine.iloc[i, -1]), axis=1).mode[:, 0]

# scikit-learnの結果と比較
knn = KNeighborsClassifier(n_neighbors=3).fit(wine.iloc[:, :-1], wine['target'])
print(f'正解率 : {sum(pred == knn.predict(wine.iloc[:, :-1])) / wine.index.size}')

正解率 : 1.0


### 次元の呪い (高次元空間内の距離)
---
$1$ 次元空間 (直線) 上では変数の値が $1$ 増えると、その間の距離は $1$ 。  
$2$ 次元空間 (平面) 上では変数の値が $1$ ずつ増えると、その間の距離は $\sqrt{2}$ 。 (元の点から距離 $1$ の点の集合は半径 $1$ の円)  
$3$ 次元空間 (空間) 上では変数の値が $1$ ずつ増えると、その間の距離は $\sqrt{3}$ 。 (元の点から距離 $1$ の点の集合は半径 $1$ の球)  
$\vdots$  
$n$ 次元空間 (超空間) 上では変数の値が $1$ ずつ増えると、その間の距離は $\sqrt{n}$ 。

以上のように、変数が増えるにしたがって、それぞれの変数のわずかな変化によってサンプル同士がいずれのサンプルからも遠くなってしまい、近いサンプルという考え方の妥当性が低くなる。これが次元の呪いの原因の一つ。

## Pythonでのk近傍法の実行方法
---
`sklearn.neighbors.KNeighborsClassifier`を使用する。使用できる距離は[`sklearn.neighbors.DistanceMetric`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html)を参照。

In [8]:
KNeighborsClassifier??

In [9]:
DistanceMetric??